## Introduction - How to use OpenWeather API

<img src="images/OpenWeather_API_Logo.jpg" width="600">


## Query Data - Making calls to the API
When you type www.google.de into your web browser and press enter, your browser will send a request to the server that is hosting google's website, asking to retrieve the contents of the website so it can display it to you. Getting data from APIs works in a similar way. A request is sent to the address of the API, but instead of returning html and javascript files that will be interpreted and rendered by you browser, almost always a JSON file with data is returned. Also while you could use a web browser to send the request, it's not needed and more convenient to use a programming language like Python to do it.

In this notebook we are going to use the requests library. It is one of the most downloaded Python packages today, pulling in around 14M downloads / week. You can find out more about it here: https://pypi.org/project/requests/.
Complete the code below and import the requests package.

In this notebook we will be finally using the OpenWeather API https://openweathermap.org/ and enrich our existing flights data with additional weather data.

In fact, you're going to:

1. Use your API key to make your first call to the OpenWeather API
2. Learn how to adjust your API calls to get the data you need
3. Learn how to access and extract data from your JSON
4. Learn how to flatten nested JSON data and transform it into a DataFrame for future analysis
5. Learn how to make multiple calls to the API with different parameters in an automated way

In [25]:
# Import requests package
import requests

Next, as mentioned above the URL of the API has to be specified. In this example we are going to pull current weather data. The documentation for this API can be found here: https://openweathermap.org/current

The url shown below is what needs to be used in order to connect to the API.

We can see that the url has 
* a fixed part: http://api.openweathermap.org/data/2.5/weather?'
* and a variable part: 'q={city name},{state code},{country_code}&appid={API key}'

<img src="images/Current_Weather_API_Call.png" width="600">

Let's take the fixed part to define a url variable.

In [26]:
# Set URL as url
url = 'http://api.openweathermap.org/data/2.5/weather?'

While the fixed part stays constant, the variable part consists of query strings or parameters, some optional some mandatory, which can be used to select or filter data or define the format or unit of measurement of the response.

Below are the optional and required parameters when pulling current weather data by city name.

<img src="images/Current_Weather_API_Call_Parameters.png" width="600">

Current weather data can be pulled not only by city name but also by

* city ID
* geographic coordinates (latitude, longitude)
* ZIP code
* cities within a rectangular zone
* cities in circle

Depending on which of the listed methods will be used, the number and names of available parameters can change. Read the documention carefully to avoid running into error messages.

In the following example we will stick with pulling the data by city name.

But first:  
We need to define a variable that includes our API key that will be used for the appid parameter. Why? Because we need to authenticate ourselves to the API when we send a request, so the API knows that we have a registered account and can notify us in case we reach our query limit.

To do this,

1. Go to you user profile and go to "My API keys" (https://home.openweathermap.org/api_keys)
2. Copy your API key.
3. Add a variable called "openweather_api_key" to your .env file and put your API key in there. Don't forget to save it!

In [27]:
# To be able to pull the API key from the .env, we need to import and run the load_dotenv function from the dotenv module first.
from dotenv import load_dotenv
load_dotenv()

True

As the final step before we can do our first API call, we have to put all necessary parameters we chose (our API key, city and units) into a dictionary. We call it "parameters", but you can choose any name.

In [28]:
# The OS module in Python provides functions for interacting with the operating system
import os

In [29]:
# complete the parameter dictionary 
parameters = {
    'appid': os.getenv('openweather_api_key'), # extract the value for the open weather api key, 
    'q': 'Düsseldorf', 
    'units': 'metric'}
#print(parameters)

We're almost ready to make our first API call!

One note: Imagine you're having multiple API calls in your script and you want to change the location parameter for all of them to the same variable at the same time. All we need to do is change the value in the dictionary and the API calls will be adjusted. This is much more convenient and less error prone compared to having to find and adjust each individual API call. This approach will also be helpful when we want to make multiple calls for e.g. different locations in one code block using loops, which we will be doing later.

Now that we have the url and the parameters, we can finally send our first API request. Since we want to get data, we need to send a GET request. This is done using the get() function from the requests function. The get() function will send a request and retrieve the response for us.

In [30]:
# Send get request
r = requests.get(url, parameters)

# Print r
print(r)

<Response [200]>


Weirdly, when we print the response, all we see is a code 200 message and no data. 

Did we do something wrong? No! 

Code 200 actually means that our request was successfull. Check out https://en.wikipedia.org/wiki/List_of_HTTP_status_codes to see a list of reponse codes and their definition. Knowing what the different response codes mean will help you understand error messages and resolve issues faster in the future.

So where is the data? The response that the API sent back is actually an object containing multiple elements such as cookies, encoding, headers and content. Run the code blocks below and check out the different elements of the response.

In [31]:
# Print headers
r.headers

{'Server': 'openresty', 'Date': 'Wed, 19 Jun 2024 09:49:52 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '471', 'Connection': 'keep-alive', 'X-Cache-Key': '/data/2.5/weather?q=d%c3%bcsseldorf&units=metric', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST'}

In [32]:
# Print cookies
r.cookies

<RequestsCookieJar[]>

In [33]:
# Print encoding
r.encoding

'utf-8'

In [34]:
# Print content
r.content

b'{"coord":{"lon":6.7762,"lat":51.2217},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04d"}],"base":"stations","main":{"temp":17.64,"feels_like":17.55,"temp_min":16.26,"temp_max":18.89,"pressure":1017,"humidity":80},"visibility":10000,"wind":{"speed":3.6,"deg":10},"clouds":{"all":75},"dt":1718790277,"sys":{"type":1,"id":1264,"country":"DE","sunrise":1718767008,"sunset":1718826708},"timezone":7200,"id":2934246,"name":"D\xc3\xbcsseldorf","cod":200}'

Found it! The content body holds the data. Since by default it's in JSON format, we can use the built-in JSON decoder to increase readability and prepare the data for future data manipulation.

In [35]:
# Apply JSON decoder and save in weather_hh
weather_hh = r.json()

# Print weather_hh
print(weather_hh)

{'coord': {'lon': 6.7762, 'lat': 51.2217}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 17.64, 'feels_like': 17.55, 'temp_min': 16.26, 'temp_max': 18.89, 'pressure': 1017, 'humidity': 80}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 10}, 'clouds': {'all': 75}, 'dt': 1718790277, 'sys': {'type': 1, 'id': 1264, 'country': 'DE', 'sunrise': 1718767008, 'sunset': 1718826708}, 'timezone': 7200, 'id': 2934246, 'name': 'Düsseldorf', 'cod': 200}


This looks good already, but we can make it look even better! To do this, we are going to use the json Python library and its functions to transform the raw data into a json object and print it with proper indentation.

In [36]:
# Import json package
import json

# Create json object: json_object
json_object = json.loads(r.content)

# Print json_object
json.dumps(json_object, indent = 3)

'{\n   "coord": {\n      "lon": 6.7762,\n      "lat": 51.2217\n   },\n   "weather": [\n      {\n         "id": 803,\n         "main": "Clouds",\n         "description": "broken clouds",\n         "icon": "04d"\n      }\n   ],\n   "base": "stations",\n   "main": {\n      "temp": 17.64,\n      "feels_like": 17.55,\n      "temp_min": 16.26,\n      "temp_max": 18.89,\n      "pressure": 1017,\n      "humidity": 80\n   },\n   "visibility": 10000,\n   "wind": {\n      "speed": 3.6,\n      "deg": 10\n   },\n   "clouds": {\n      "all": 75\n   },\n   "dt": 1718790277,\n   "sys": {\n      "type": 1,\n      "id": 1264,\n      "country": "DE",\n      "sunrise": 1718767008,\n      "sunset": 1718826708\n   },\n   "timezone": 7200,\n   "id": 2934246,\n   "name": "D\\u00fcsseldorf",\n   "cod": 200\n}'

Awesome! Now it's super easy to identify the different key-value pairs and assess the available information.

You might be saying: "Yeah, this is cool and all, but I'm not a weather expert, what do all these fields mean?" 

-> **Read the documention!**

Go to https://openweathermap.org/current#current_JSON and you will find a list of all fields and their definition that can be found in our JSON response.

As mentioned above, we can see the different key-value pairs inside the JSON output. You've probably heard the term key-value pairs before. Do you remember where? Dictionaries, exactly! If you need a quick refresh on what dictionaries are and what they do, check out the official Python documentation here: https://docs.Python.org/3/tutorial/datastructures.html#dictionaries.

Printing the type of our weather_hh variable, which is the API response decoded by the JSON decoder, we get the confirmation that we are working with a dictionary.

In [37]:
# Print type of weather_hh
print(type(weather_hh))

<class 'dict'>


We can use different techniques to access all or only specific parts of the data. One way is to loop through the dictionary and print all key, value pairs.  
Complete the code below and print all key value pairs.

In [38]:
# Loop through and print all key-value pairs
for key, value in weather_hh.items():
    print(key + ':', value)

coord: {'lon': 6.7762, 'lat': 51.2217}
weather: [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}]
base: stations
main: {'temp': 17.64, 'feels_like': 17.55, 'temp_min': 16.26, 'temp_max': 18.89, 'pressure': 1017, 'humidity': 80}
visibility: 10000
wind: {'speed': 3.6, 'deg': 10}
clouds: {'all': 75}
dt: 1718790277
sys: {'type': 1, 'id': 1264, 'country': 'DE', 'sunrise': 1718767008, 'sunset': 1718826708}
timezone: 7200
id: 2934246
name: Düsseldorf
cod: 200


Alternatively, we can print specific values by referencing their respective key.

In [42]:
# Print values in base
#weather_hh['base'] 

# Print values in main
# weather_hh['main']

# Print values in temp
weather_hh['main']['temp']



17.64

## Working with (nested) JSONs
Now that you've learned how to pull current weather data from the OpenWeather API and explore the output, it's time to convert it to a DataFrame. Having the data in a DataFrame allows us to perform data exploration, manipulation or visualization.

The DataFrame() function in the pandas package allows us to transform a dictionary into a DataFrame.
Run the code below to import the library and transform the current weather in Hamburg into a DataFrame.

In [46]:
# Import pandas package

import pandas as pd

# Transform dictionary to dataframe (will throw an error)
#pd.DataFrame(weather_hh)

#ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

Weird, we're getting an error message. Do you have an idea what could be the issue?

Don't worry if you have no clue what the problem is. Let's look at our JSON output again. Execute the code below.

In [47]:
# Print content of json_object
json.dumps(json_object, indent = 3)

'{\n   "coord": {\n      "lon": 6.7762,\n      "lat": 51.2217\n   },\n   "weather": [\n      {\n         "id": 803,\n         "main": "Clouds",\n         "description": "broken clouds",\n         "icon": "04d"\n      }\n   ],\n   "base": "stations",\n   "main": {\n      "temp": 17.64,\n      "feels_like": 17.55,\n      "temp_min": 16.26,\n      "temp_max": 18.89,\n      "pressure": 1017,\n      "humidity": 80\n   },\n   "visibility": 10000,\n   "wind": {\n      "speed": 3.6,\n      "deg": 10\n   },\n   "clouds": {\n      "all": 75\n   },\n   "dt": 1718790277,\n   "sys": {\n      "type": 1,\n      "id": 1264,\n      "country": "DE",\n      "sunrise": 1718767008,\n      "sunset": 1718826708\n   },\n   "timezone": 7200,\n   "id": 2934246,\n   "name": "D\\u00fcsseldorf",\n   "cod": 200\n}'

We can see that some key-value pairs have further key-value pairs as values. The key "weather" for example has four key value pairs nested within: "id", "main", "description", "icon". The DataFrame() function expects dictionaries with only one level of key-value pairs, but since we have nested pairs, it throws an error.

There are multiple solutions to this problem and the one that will be the most useful to you depends on what parts of the data you ultimately need for your analysis. We need all data in our output and therefore the method we are going to use is to flatten the JSON data using the json_normalize() function from the pandas package. It flattens our JSON data and transforms it into a DataFrame. Execute the code below and check the output.

In [48]:
# Flatten json and save in weather_hh_norm
weather_hh_norm = pd.json_normalize(weather_hh, sep="_")

# Print weather_hh_norm

print(weather_hh_norm)

                                             weather      base  visibility  \
0  [{'id': 803, 'main': 'Clouds', 'description': ...  stations       10000   

           dt  timezone       id        name  cod  coord_lon  coord_lat  ...  \
0  1718790277      7200  2934246  Düsseldorf  200     6.7762    51.2217  ...   

   main_pressure  main_humidity  wind_speed  wind_deg  clouds_all  sys_type  \
0           1017             80         3.6        10          75         1   

   sys_id  sys_country  sys_sunrise  sys_sunset  
0    1264           DE   1718767008  1718826708  

[1 rows x 24 columns]


Awesome! Now we have a DaraFrame with 1 row and 24 columns. But wait, for some reason the "weather" column still contains data in JSON format. This is because the nested key-value pairs are in a list. Probably because sometimes the weather column can be nested further. In order to resolve this we need to use the advanced parameters of the json_normalize() function.

* record_path = specify the key that is nested deeper
* meta = specify the structure of the remaining JSON
* record_prefix = adds a prefix to the column names in the record_path to avoid duplicate column names

Execute the code below and compare the output to the output above.

In [49]:
# Advanced flattening with json_normalize()
weather_hh_df = pd.json_normalize(weather_hh, 
                                  sep="_", 
                                  record_path="weather", 
                                  meta=[["coord", "lon"], 
                                        ["coord", "lat"], 
                                        "base",
                                        ["main", "temp"],
                                        ["main", "feels_like"],
                                        ["main", "temp_min"],
                                        ["main", "temp_max"], 
                                        ["main", "pressure"], 
                                        ["main", "humidity"], 
                                        "visibility", 
                                        ["wind", "speed"], 
                                        ["wind", "deg"], 
                                        ["clouds", "all"], 
                                        "dt", 
                                        ["sys", "type"], 
                                        ["sys", "id"],
                                        ["sys", "country"],
                                        ["sys", "sunrise"],
                                        ["sys", "sunset"],
                                        "timezone",
                                        "id",
                                        "name",
                                        "cod"], 
                                  record_prefix="weather_")
# Print weather_hh_df
print(weather_hh_df)

   weather_id weather_main weather_description weather_icon coord_lon  \
0         803       Clouds       broken clouds          04d    6.7762   

  coord_lat      base main_temp main_feels_like main_temp_min  ...  \
0   51.2217  stations     17.64           17.55         16.26  ...   

           dt sys_type sys_id sys_country sys_sunrise  sys_sunset timezone  \
0  1718790277        1   1264          DE  1718767008  1718826708     7200   

        id        name  cod  
0  2934246  Düsseldorf  200  

[1 rows x 27 columns]


You're probably thinking: "Wow, this requires a lot of manual work typing all those keys in the paramaters". And yes, you're right. Unfortunately this is one of the easier methods to solve the issue. Also, it's not perfect and comes with its own limitations. If you have data that is nested even deeper you often have no other choice but to write your own flattening function. Fortunately there are other Python specialists out there who have done so and shared their work here: 

* https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
* https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame
* https://medium.com/swlh/converting-nested-json-structures-to-pandas-dataframes-e8106c59976e


Alright, now that the data is in the right format, we don't have to worry about nested JSONs anymore. To make our data more interesting, since right now we only have one row, let's pull current weather data from more cities and create a proper table with multiple rows. In order to do that, we're going to combine, extend and apply everything we've learnt so far:

* Define the parameters in the request URL
* Send a request and retrieve the response using the get() function
* Decode the response using the JSON decoder .json()
* Flatten the JSON file and transform it into a DataFrame

The last step will be to combine all dataframes into one final table.

This time we want to get the current weather data of multiple locations. Unfortunately the location parameter "q" only takes one location at a time. Using a for-loop we should be able to make multiple API calls while iterating through several locations.

First, let's collect all the pieces we need below and

1. Change the location variable to locations and have to include a list with multiple locations
2. Define an empty DataFrame variable
3. Add a for-loop that iterates through all locations, flattens the JSON output and appends it to the DataFrame

**IMPORTANT: Don't run the code below too often per minute, since you only have 60 API calls per minute you can quickly run into a temporary query limit!**

In [53]:
# Set the url
url = 'http://api.openweathermap.org/data/2.5/weather?'

# Set locations
locations = ["q=Hamburg", 
             "q=Berlin", 
             "q=London", 
             "q=Madrid",
             "q=New York",
             "q=Moscow",
             "q=New York",
             "q=Ankara",
             "q=Baghdad",
             "q=Kabul",
             "q=Tokio",
             "q=Taipei",
             "q=Manila",
             "q=Auckland",
             "q=Seoul"]

# Set unit of measurement
unit = "&units=metric"

# Set API key (defined already)
api_key = '&appid=' + os.getenv('openweather_api_key')

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])

# Loop through all locations
for location in locations:
    # Create final url
    url_f = url + location + unit + api_key
    
    # Request data from url
    r = requests.get(url_f)
    
    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = r.json()

    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", errors='ignore',
                                        record_path="weather", 
                                        meta=[["coord", "lon"], 
                                              ["coord", "lat"], 
                                              "base",
                                              ["main", "temp"],
                                              ["main", "feels_like"],
                                              ["main", "temp_min"],
                                              ["main", "temp_max"], 
                                              ["main", "pressure"], 
                                              ["main", "humidity"], 
                                              "visibility", 
                                              ["wind", "speed"], 
                                              ["wind", "deg"], 
                                              ["clouds", "all"], 
                                              "dt", 
                                              ["sys", "type"], 
                                              ["sys", "id"],
                                              ["sys", "country"],
                                              ["sys", "sunrise"],
                                              ["sys", "sunset"],
                                              "timezone",
                                              "id",
                                              "name",
                                              "cod"], 
                                        record_prefix="weather_")
    
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df

,weather_id,weather_main,weather_description,weather_icon,coord_lon,coord_lat,base,main_temp,main_feels_like,main_temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod
0,803,Clouds,broken clouds,04d,10,53.55,stations,17.66,17.18,16.08,...,1718792858,1,1263,DE,1718765406,1718826763,7200,2911298,Hamburg,200
1,803,Clouds,broken clouds,04d,13.4105,52.5244,stations,17.77,17.69,16.66,...,1718792729,2,2011538,DE,1718764968,1718825564,7200,2950159,Berlin,200
2,803,Clouds,broken clouds,04d,-0.1257,51.5085,stations,17.26,16.71,14.92,...,1718792702,2,2093698,GB,1718768569,1718828461,3600,2643743,London,200
3,802,Clouds,scattered clouds,03d,-3.7026,40.4165,stations,19.74,19.34,18.48,...,1718792239,2,2084029,ES,1718772268,1718826479,7200,3117735,Madrid,200
4,801,Clouds,few clouds,02d,-74.006,40.7143,stations,21.55,21.77,19.12,...,1718792590,1,4610,US,1718789083,1718843415,-14400,5128581,New York,200
5,803,Clouds,broken clouds,04d,37.6156,55.7522,stations,26.04,26.04,24.1,...,1718792493,2,2095214,RU,1718757860,1718821051,10800,524901,Moscow,200
6,801,Clouds,few clouds,02d,-74.006,40.7143,stations,21.55,21.77,19.12,...,1718792590,1,4610,US,1718789083,1718843415,-14400,5128581,New York,200
7,802,Clouds,scattered clouds,03d,32.8543,39.9199,stations,30.06,28.81,29.51,...,1718792575,2,267643,TR,1718763591,1718817606,10800,323786,Ankara,200
8,800,Clear,clear sky,01d,44.4009,33.3406,stations,46.95,42.21,46.95,...,1718792985,1,7597,IQ,1718761980,1718813674,10800,98182,Baghdad,200
9,800,Clear,clear sky,01d,69.4167,34.5,stations,29.68,27.88,29.68,...,1718793006,NaN,NaN,AF,1718755787,1718807858,16200,1138957,Kabul,200


Fantastic! Now we have a DataFrame with current weather information for 15 locations. What's even better, we didn't have to write a lot of complicated code. How cool is that!?  

If we wanted to, we could now create a new weather table in our database and write the weather data into it. Guess what, this is exactly what you're going to do in the upcoming project. So take a break and then move on to the next challenge, good luck!

### Extra Credit 

If you have time and energy, we have an additional challenge for you.

Until now, you got to know two different methods to call an API:
1. Pass the general part of the URL and put the rest of the information into a dictionary and pass it as an additional argument.
<br>`r = requests.get(url_part, parameters)`
2. Join together all parts of the URL as strings and pass the whole URL to the get function.
<br>`url_all = url_part + location + unit + api_key`
<br>`r = requests.get(url_all)`

Why don't you apply the first method (passing parameters to the get function) to the task in which you used the second method. <br> Can you do it?
